# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Tables](#tables)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb"
    ]
}
```

# **TABLES**

---

### STAGES

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[MUNICIPALITY_RWZI_COUPLING]'))
CREATE TABLE [VWSSTAGE].[MUNICIPALITY_RWZI_COUPLING] (
	[ID] [INT] PRIMARY KEY IDENTITY (1,1),
	[DATE_LAST_INSERTED] [DATETIME] NOT NULL DEFAULT GETDATE(),
	[VERSIE] [VARCHAR](256) NOT NULL,
	[DATUM_BESTAND] [VARCHAR](256) NOT NULL,
	[DATUM_HERINDELING] [VARCHAR](256) NOT NULL,
	[REGIO_CODE] [VARCHAR](256) NOT NULL,
	[REGIO_CODE_INT] [VARCHAR](256) NOT NULL,
	[REGIO_NAAM] [VARCHAR](256) NOT NULL,
	[RWZI_CODE] [VARCHAR](256) NOT NULL,
	[RWZI_NAAM] [VARCHAR](256) NOT NULL,
	[ACTIEF] [VARCHAR](256) NOT NULL
);
GO

### STATICS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTATIC].[MUNICIPALITY_RWZI_COUPLING]'))
CREATE TABLE [VWSSTATIC].[MUNICIPALITY_RWZI_COUPLING] (
	[ID] [INT] PRIMARY KEY IDENTITY (1,1),
	[DATE_LAST_INSERTED] [DATETIME] NOT NULL DEFAULT GETDATE(),
	[VERSIE] INT NOT NULL,
	[DATUM_BESTAND] [DATETIME] NOT NULL,
	[DATUM_HERINDELING] [DATETIME] NOT NULL,
	[REGIO_CODE] [VARCHAR](256) NOT NULL,
	[REGIO_CODE_INT] INT NOT NULL,
	[REGIO_NAAM] [VARCHAR](256) NOT NULL,
	[RWZI_CODE] INT NOT NULL,
	[RWZI_NAAM] [VARCHAR](256) NOT NULL,
	[ACTIEF] BIT NOT NULL
);
GO

# **STORED PROCEDURES**

---


### STAGING &rarr; STATIC MAPPING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE OR ALTER PROCEDURE [DBO].[SP_MUNICIPALITY_RWZI_COUPLING_STAGE_TO_STATIC]
AS
BEGIN

INSERT INTO [VWSSTATIC].[MUNICIPALITY_RWZI_COUPLING] (
    [VERSIE],
    [DATUM_BESTAND],
    [DATUM_HERINDELING],
    [REGIO_CODE],
    [REGIO_CODE_INT],
    [REGIO_NAAM],
    [RWZI_CODE],
    [RWZI_NAAM],
    [ACTIEF]
    )
SELECT
    [VERSIE],
    CONVERT(DATETIME, [DATUM_BESTAND], 102),
    CONVERT(DATETIME, [DATUM_HERINDELING], 102),
    [REGIO_CODE],
    [REGIO_CODE_INT],
    [REGIO_NAAM],
    [RWZI_CODE],
    [RWZI_NAAM],
    [ACTIEF]
FROM [VWSSTAGE].[MUNICIPALITY_RWZI_COUPLING]
WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[MUNICIPALITY_RWZI_COUPLING])
END;
GO

# **DATATINO CONFIGURATIONS**

---

### WORKFLOWS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) UPSERT WORKFLOW(S).....
DECLARE @workflow_name NVARCHAR(50) = 'STATIC_DATA',
        @workflow_id INT,        
        @workflow_description VARCHAR(256),
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @workflow_id = NULL;

SELECT TOP(1)
    @workflow_id = workflows.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
WHERE v_workflows.[NAME] = @workflow_name;

SET @workflow_description = CONCAT('WORKFLOW: ', @workflow_name);

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
    @id = @workflow_id, 
    @workflow_name = @workflow_name,
    @description = @workflow_description,
    @SCHEDULE = '0 14 * */3 MON-FRI', -- AT 12:00 PM, EVERY 3 MONTHS
    @active = @is_active;

-- 2) UPSERT SOURCE(S).....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256),
        @target_name VARCHAR(256),
        @location_type VARCHAR(50),
        @security_profile VARCHAR(50);

-- 2.1) SET ENVIRONMENTAL VARIABLES.....
SET @source = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'datafiles/STATIC/Gemeenten_RWZI_Koppeling.csv'
    WHEN 'acceptance' THEN 'datafiles/STATIC/Gemeenten_RWZI_Koppeling.csv'
    ELSE 'datafiles/STATIC/Gemeenten_RWZI_Koppeling.csv'
END;

SET @location_type = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'AzureBlob'
    WHEN 'acceptance' THEN 'AzureBlob'
    ELSE 'AzureBlob'
END;

SET @security_profile = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'N/A'
    WHEN 'acceptance' THEN 'N/A'
    ELSE 'N/A'
END;

-- 2.2) UPSERT TABLE SOURCE(S): STAGING......
SET @source_id = NULL;
SET @source_name = 'SOURCE_MUNICIPALITY_RWZI_COUPLING_STATIC_DATA';
SET @source_description = CONCAT('LOAD: ', @workflow_name);
SET @target_name = 'VWSSTAGE.MUNICIPALITY_RWZI_COUPLING';

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = 'versie|datum_bestand|datum_herindeling|regio_code|regio_code_int|regio_naam|rwzi_code|rwzi_naam|actief',
    @target_columns = 'VERSIE|DATUM_BESTAND|DATUM_HERINDELING|REGIO_CODE|REGIO_CODE_INT|REGIO_NAAM|RWZI_CODE|RWZI_NAAM|ACTIEF',
    @target_name = @target_name,
    @source_type = 'CsvFile',
    @location_type = @location_type,
    @delimiter_type = 'SemiColon',
    @security_profile= @security_profile;

-- 2.3) UPSERT STORED PROCEDURE SOURCE(S): STAGE TO STATIC......
SET @source_id = NULL;
SET @source = 'dbo.SP_MUNICIPALITY_RWZI_COUPLING_STAGE_TO_STATIC';
SET @source_name = 'SOURCE_SP_MUNICIPALITY_RWZI_COUPLING_STAGE_TO_STATIC';
SET @source_description = 'MAP: MUNICIPALITY_RWZI_COUPLING FROM STAGE TO STATIC';

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= @security_profile;

-- 3) UPSERT PROCESS(ES).....
DECLARE @process_id INT,
        @process_name NVARCHAR(256),
        @process_description NVARCHAR(256),
        @process_source_name VARCHAR(256);

-- 3.1) UPSERT TABLE PROCESS(ES): STAGING......
SET @process_id = NULL;
SET @process_name = 'PROCESS_MUNICIPALITY_RWZI_COUPLING_STATIC_DATA';
SET @process_description = CONCAT('LOAD: ', @workflow_name);
SET @process_source_name = 'SOURCE_MUNICIPALITY_RWZI_COUPLING_STATIC_DATA';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 3.2) UPSERT STORED PROCEDURE PROCESS(S): STAGE TO STATIC......
SET @process_id = NULL;
SET @process_name = 'PROCESS_SP_MUNICIPALITY_RWZI_COUPLING_STAGE_TO_STATIC';
SET @process_description = 'MAP: MUNICIPALITY_RWZI_COUPLING FROM STAGE TO STATIC';
SET @process_source_name = 'SOURCE_SP_MUNICIPALITY_RWZI_COUPLING_STAGE_TO_STATIC';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 4) UPSERT DEPENDENC(Y)(IES).....
DECLARE @dependency_id INT,
        @dependency_name NVARCHAR(256),
        @dependency_description NVARCHAR(256),
        @dependency_dataflow_name NVARCHAR(256),
        @dependency_process_name NVARCHAR(256);

-- 4.1) UPSERT STAGING TO STORE PROCEDURE DEPENDENC(Y)(IES)....
SET @dependency_id = NULL;
SET @dependency_name = 'DEPENDENCY_MUNICIPALITY_RWZI_COUPLING_STAGE_TO_STATIC';
SET @dependency_dataflow_name = 'PROCESS_SP_MUNICIPALITY_RWZI_COUPLING_STAGE_TO_STATIC';
SET @dependency_process_name = 'PROCESS_MUNICIPALITY_RWZI_COUPLING_STATIC_DATA';
SET @dependency_description = CONCAT('TRIGGER ', @dependency_dataflow_name,' AFTER ', @dependency_process_name ,' HAS FINISHED');

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;
GO